<a href="https://colab.research.google.com/github/igordantasgf/PythonFrameworks/blob/main/Python%20e%20BDs%20Relacionais/py_sql_mongo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **ORM : Object Relational Mapper**
RM ou mapeador objeto relaciona, é um tipo de ferramenta muito utilizada hoje em dia, com o propósito de unir o mundo orietado a objetos e o mundo relacional.
#### Objeto $→$ Modelo Relacional

#### **Vantagens:**
- Menos código
- Melhor manutenção
- Utilização de conectores
- Indicado para CRUDs (é um acrônimo para as quatro basic operations: creeate, read, update, delete)

### **Por que usar?**
- Troca de SGBD mais facilitada (Sistema de Gerenciamento de Banco de Dados)
- Modelo MVC (Model-View-Controller)
- Diminuição do DRY (Don't repeat yourself)
- Evita problemas de segurança

In [5]:
from IPython.display import clear_output
!pip install pymongo
clear_output()

In [6]:
import sqlalchemy
import pymongo